<a href="https://colab.research.google.com/github/adaryass/Modern-Portfolio-Theory-Markowitz-Model-/blob/main/Stock_Portfolio_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import scipy.optimize as optimisation


# On average there are 252 trading days in year
NUM_TRADING_DAYS = 252
# We will generate random  w (different portfolios)
NUM_PORTFOLIOS = 10000

# Stock we are going to handle
stocks = ['AAPL', 'WMT', 'TSLA', 'GE', 'AMZN', 'DB']

# Historical data - define START and END dates
start_date = '2012-01-01'
end_date = '2025-01-01'

def download_data():
  # Name of the stock (key) -stock values (2012-2017) as the value
  stock_data = {}
  for stock in stocks:
    # Closing prices
    ticker = yf.Ticker(stock)
    stock_data[stock] = ticker.history(start=start_date, end=end_date)['Close']

  return pd.DataFrame(stock_data)

def show_data(data):
  fig = px.line(data, title='Stock Prices Over Time')
  fig.show()


def calculate_return(data):
  log_return = np.log(data / data.shift(1))
  return log_return[1:]

def show_statistics(returns):
  # Instead of daily metrics we are after annual metrics
  # Mean of annual return
  print("Mean of annual return")
  print(returns.mean() * NUM_TRADING_DAYS)
  print('\n')
  print("Matix Covariance annual")
  print(returns.cov()*  NUM_TRADING_DAYS)

def show_mean_variance(returns, weights):
  portfolio_return = np.sum(returns.mean() * weights) * NUM_TRADING_DAYS
  portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() *
                                                          NUM_TRADING_DAYS, weights)))
  print(portfolio_return)
  print(portfolio_volatility)

def show_portfolios(returns, volatilities):
  fig = go.Figure(data=go.Scatter(x=volatilities, y=returns, mode='markers',
                                  marker=dict(color=returns/volatilities, colorscale='Viridis', showscale=True,
                                              colorbar=dict(title='Sharpe Ratio'))))
  fig.update_layout(title='Portfolio Optimization',
                    xaxis_title='Expected Volatility',
                    yaxis_title='Expected Return')
  fig.show()

def generate_portfolios(returns):

  portfolios_means = []
  portfolio_risks = []
  portfolio_weights = []

  for _ in range(NUM_PORTFOLIOS):
    w = np.random.random(len(stocks))
    w /= np.sum(w)
    portfolio_weights.append(w)
    portfolios_means.append(np.sum(returns.mean() * w) * NUM_TRADING_DAYS)
    portfolio_risks.append(np.sqrt(np.dot(w.T, np.dot(returns.cov() *
                                                      NUM_TRADING_DAYS, w))))

  return np.array(portfolio_weights), np.array(portfolios_means), np.array(portfolio_risks)


def statistics(weights, returns):
  portfolio_return = np.sum(returns.mean() * weights) * NUM_TRADING_DAYS
  portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() *
                                                          NUM_TRADING_DAYS, weights)))
  return np.array([portfolio_return, portfolio_volatility,
                   portfolio_return/portfolio_volatility])

# Scipy optimize module can find the minimum of a given function
# The maximum of a f(x) is the minimum of -f(x)
def min_func_sharpe(weights, returns):
  return -statistics(weights, returns)[2]

# What are the constraints ? The sum of weights =1 !!
# f(x)=0 this is the function to minimise
def optimize_portfolio(weights, returns):
  # The sum of weights is 1
  constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
  # The weights can be 1 at most: 1 when 100% of money is invested into a single stock
  bounds =tuple((0, 1) for _ in range(len(stocks)))
  return optimisation.minimize(fun=min_func_sharpe, x0=weights[0], args=returns,
                                method='SLSQP', bounds=bounds,
                                constraints=constraints)

def print_optimal_portfolio(optimum, returns):
  print('Optimal portfolio: ', optimum['x'].round(3))
  print('Expected return, volatility and Sharpe ratio: ',
        statistics(optimum['x'].round(3), returns))

def show_optimal_portfolio(opt, rets, portfolio_rets, portfolio_vols):
  fig = go.Figure(data=go.Scatter(x=portfolio_vols, y=portfolio_rets, mode='markers',
                                  marker=dict(color=portfolio_rets/portfolio_vols, colorscale='Viridis', showscale=True,
                                              colorbar=dict(title='Sharpe Ratio'))))
  fig.add_trace(go.Scatter(x=[statistics(opt['x'], rets)[1]], y=[statistics(opt['x'], rets)[0]],
                           mode='markers', marker=dict(color='green', size=20, symbol='star'),
                           name='Optimal Portfolio'))
  fig.update_layout(title='Portfolio Optimization with Optimal Portfolio',
                    xaxis_title='Expected Volatility',
                    yaxis_title='Expected Return')
  fig.show()



if __name__ == '__main__':

  dataset = download_data()
  show_data(dataset)
  calculate_return(dataset)
  log_daily_return = calculate_return(dataset)
  # show_statistics(log_daily_return)

  pweights, means, risks = generate_portfolios(log_daily_return)
  show_portfolios(means, risks)
  optimum = optimize_portfolio(pweights, log_daily_return)
  print_optimal_portfolio(optimum, log_daily_return)
  show_optimal_portfolio(optimum, log_daily_return, means, risks)

Optimal portfolio:  [0.262 0.419 0.13  0.    0.189 0.   ]
Expected return, volatility and Sharpe ratio:  [0.2185441  0.20110062 1.08674007]
